## The Augmented LLM
In this workbook we'll build a sample workflow of an Augmented LLM, like the one shown in the image below:

<div align="center">
<img src="images/01-Augmented-LLM.png" width="300" heigh="250" alt="Augmented LLM"/>
</div>

In [2]:
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langchain.chat_models import init_chat_model

In [3]:
load_dotenv(override=True)

True

In [4]:
# create our LLM - we'll be using Google Gemini flash
llm = init_chat_model("google_genai:gemini-2.5-flash", temperature=0.0)

/home/mjbhobe/code/git-projects/learning_langgraph/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# for structured results from query, we use a Pydantic model
class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Query that is an optimized web search")
    justification: str = Field(
        None, description="Why this query is relevant to user's request"
    )

# augment llm with structured output
structured_llm = llm.with_structured_output(SearchQuery)

In [6]:
# ask LLM a question
response = structured_llm.invoke(
    "Does regular consumption of rice, in resonable proportions, cause weight gain?"
)
print(f"Search Query: {response.search_query}")
print(f"Justification: {response.justification}")

Search Query: does regular consumption of rice in reasonable proportions cause weight gain
Justification: The user is asking about the effect of regular rice consumption on weight gain when consumed in reasonable proportions. A web search can provide information on this topic.


Now let us illustrate how you can use tool(s) with an LLM.

In [7]:
# define the tool (basically just a function)
def multiply(a: float, b: float) -> float:
    """return the product or multiplication of two numbers"""
    return a * b


# bind the LLM to tools provided
llm_with_tools = llm.bind_tools([multiply])

# and call the LLM
query: str = "What is product of 45 and 6"
response = llm_with_tools.invoke(query)
print(f"{query}: {response.content}")
# check which tools, if any, were called by LLM
print(response.tool_calls)

What is product of 45 and 6: 
[{'name': 'multiply', 'args': {'a': 45.0, 'b': 6.0}, 'id': 'a7d3d516-4b1a-4d46-bdab-11abdad3a6af', 'type': 'tool_call'}]


In [8]:
response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 45.0, "b": 6.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--440c660e-4ac1-40a8-8776-3000fe58bb62-0', tool_calls=[{'name': 'multiply', 'args': {'a': 45.0, 'b': 6.0}, 'id': 'a7d3d516-4b1a-4d46-bdab-11abdad3a6af', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 19, 'total_tokens': 146, 'input_token_details': {'cache_read': 0}})

Notice that the LLM _does not_ actually call the tool. It just tells you that "for this user query _you_ should be calling _this specific tool_" (in our case that'll be `multiply`). The LLM _helps_ you with the payload that should be used for the tool call (in this case `{'a': 45.0 and 'b': 6.0}` - perfect!). Then you should call the LLM again with this payload and you get an _Agent_!